In [1]:
import gradio as gr

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['font.sans-serif'] = ['KaiTi']  #指定默认字体 SimHei黑体
plt.rcParams['axes.unicode_minus'] = False   #解决保存图像是负号'
import jieba
stop_list  = pd.read_csv('E:\停用词表\stopwords-master\cn_stopwords.txt',index_col=False,quoting=3,
                         sep="\t",names=['stopword'], encoding='utf-8')

In [3]:
#Jieba分词函数
def txt_cut(juzi):
    lis=[w for w in jieba.lcut(juzi) if w not in stop_list.values]
    return " ".join(lis)
 
df=pd.read_csv("D:\\python data\\第二轮考核\\data\\news_train.csv",encoding='gb18030')
data=pd.DataFrame()
data['label']=df['标签']
data['cutword']=df['新闻'].astype('str').apply(txt_cut)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\yuan\AppData\Local\Temp\jieba.cache
Loading model cost 0.376 seconds.
Prefix dict has been built successfully.


In [4]:
from os import listdir
from keras.preprocessing import sequence
import keras.preprocessing
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
# 将文件分割成单字, 建立词索引字典     
tok = Tokenizer(num_words=6000)
tok.fit_on_texts(data['cutword'].values)

In [5]:
# 将序列数据填充成相同长度 
X= tok.texts_to_sequences(data['cutword'].values)
from tensorflow import keras 
from keras.preprocessing import sequence
X= keras.preprocessing.sequence.pad_sequences(X, maxlen=90)
print(X.shape)
Y=data['label'].values
print(Y.shape)

(10000, 90)
(10000,)


In [6]:
Y = to_categorical(Y)

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Reshape,Flatten,Dense,Dropout,Input,Conv2D,MaxPool2D,concatenate
filter_size=[3,4,5]
def convolution():
    inn=Input(shape=(90,100,1))
    cnns=[]
    for size in filter_size:
        conv=Conv2D(64,(size,100),
                           strides=1,padding='valid',activation='relu')(inn)
        #将64个特征池化
        pool=MaxPool2D(pool_size=(90-size+1,1),padding='valid')(conv)
        cnns.append(pool)
        
        #将所有特征图拼在一起
    outt=concatenate(cnns)
        
    model=keras.Model(inputs=inn,outputs=outt)
    return model
    

def cnn_mulfilter():
    model=Sequential()
    model.add(Embedding(input_dim=6000,output_dim=100,input_length=90))
    model.add(Reshape((90,100,1)))
    model.add(convolution())
    model.add(Flatten())
    model.add(Dense(11,activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(11,activation='sigmoid'))
    
    model.compile(optimizer=keras.optimizers.Adam(),
                 loss=keras.losses.BinaryCrossentropy(),
                 metrics=['accuracy'])
    return model

model=cnn_mulfilter()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 90, 100)           600000    
                                                                 
 reshape (Reshape)           (None, 90, 100, 1)        0         
                                                                 
 model (Functional)          (None, 1, 1, 192)         76992     
                                                                 
 flatten (Flatten)           (None, 192)               0         
                                                                 
 dense (Dense)               (None, 11)                2123      
                                                                 
 dropout (Dropout)           (None, 11)                0         
                                                                 
 dense_1 (Dense)             (None, 11)                1

In [8]:
history = model.fit(X,Y,batch_size=64,epochs=10,validation_split=0.1)

Epoch 1/10
141/141 [==============================] - 6s 33ms/step - loss: 0.4406 - accuracy: 0.1641 - val_loss: 0.2315 - val_accuracy: 0.3730
Epoch 2/10
141/141 [==============================] - 4s 32ms/step - loss: 0.2068 - accuracy: 0.5859 - val_loss: 0.1065 - val_accuracy: 0.8640
Epoch 3/10
141/141 [==============================] - 4s 31ms/step - loss: 0.1330 - accuracy: 0.7414 - val_loss: 0.0731 - val_accuracy: 0.8970
Epoch 4/10
141/141 [==============================] - 5s 32ms/step - loss: 0.0977 - accuracy: 0.8301 - val_loss: 0.0574 - val_accuracy: 0.9120
Epoch 5/10
141/141 [==============================] - 4s 32ms/step - loss: 0.0797 - accuracy: 0.8704 - val_loss: 0.0526 - val_accuracy: 0.9180
Epoch 6/10
141/141 [==============================] - 4s 31ms/step - loss: 0.0676 - accuracy: 0.8958 - val_loss: 0.0504 - val_accuracy: 0.9200
Epoch 7/10
141/141 [==============================] - 4s 32ms/step - loss: 0.0588 - accuracy: 0.9078 - val_loss: 0.0506 - val_accuracy: 0.9220

In [9]:
# 保存模型
model.save('E:\\model_stored.h5')
 
# 加载模型，同时加载了模型的结构、权重等信息
new_model = keras.models.load_model('E:\\model_stored.h5')

In [10]:
# test='基金风险准备金最低提取比例翻番早报讯 中国证监会5日发布公告，将基金管理公司风险准备金的最低计提比例由原来管理费收入的5%提高到10%，以进一步增强基金管理公司的风险防范能力，增强基金份额持有人的信心，保护基金份额持有人的利益，促进基金行业持续稳健发展。根据修改后的规定，基金管理公司应当每月从基金管理费收入中计提风险准备金，计提比例不低于基金管理费收入的10%。风险准备金余额达到基金资产净值的1%时可以不再提取。(新华）'

In [11]:
import numpy as np
import pandas as pd
from tensorflow import keras 
from keras.preprocessing import sequence

In [12]:
# test=pd.Series(test)

In [13]:
# test1=pd.DataFrame()
# test1['新闻']=test
# test1

In [14]:
# test_x=pd.DataFrame()
# test_x['cutword']=test1['新闻'].astype('str').apply(txt_cut)
# test_x

In [15]:
# #词填充
# test_X= tok.texts_to_sequences(test_x['cutword'].values)
# test_X= keras.preprocessing.sequence.pad_sequences(test_X, maxlen=90)
# test_X

In [16]:
# predictions=model.predict(test_X)
# print('输出结果：\n',predictions) 

In [17]:
# def trans_to_onehot(X):
#     result=[]
#     for i in range(len(X)):
#         max=0
#         max_index=0
#         result2=[]
#         for j in range(len(X[i])):
#             if X[i][j]>max:
#                 max=X[i][j]
#                 max_index=j
#         for k in range(len(X[i])):
#             if k !=max_index:
#                 result2.append(0)
#             else:
#                 result2.append(1)

#         result.append(result2)
#     return result
# result=trans_to_onehot(predictions)
# result

In [18]:
# lebal = [np.argmax(one_hot)for one_hot in result]
# lebal

In [19]:
def predict(input_w):
    test=pd.Series(input_w)
    test1=pd.DataFrame()
    test1['新闻']=test
    test_x=pd.DataFrame()
    test_x['cutword']=test1['新闻'].astype('str').apply(txt_cut)
    # #词填充
    test_X= tok.texts_to_sequences(test_x['cutword'].values)
    test_X= keras.preprocessing.sequence.pad_sequences(test_X, maxlen=90)
    predictions=model.predict(test_X)
    
    def trans_to_onehot(X):
        result=[]
        for i in range(len(X)):
            max=0
            max_index=0
            result2=[]
            for j in range(len(X[i])):
                if X[i][j]>max:
                    max=X[i][j]
                    max_index=j
            for k in range(len(X[i])):
                if k !=max_index:
                    result2.append(0)
                else:
                    result2.append(1)

        result.append(result2)
        return result
    result=trans_to_onehot(predictions)
    
    lebal = [np.argmax(one_hot)for one_hot in result]

    dic={1:'娱乐',2:'财经', 3:'时尚', 4:'房产', 5:'游戏', 6:'科技', 7:'家居', 8:'时政', 9:'教育', 10:'体育'}

    a=lebal[0]
    lebal=dic[a]
    
    return lebal

In [20]:
# dic={'娱乐':1, '财经':2, '时尚':3, '房产':4, '游戏':5, '科技':6, '家居':7, '时政':8, '教育':9, '体育':10}
# dic={'1':'娱乐', '2':'财经', '3':'时尚', '4':'房产', '5':'游戏', '6':'科技', '7':'家居', '8':'时政', '9':'教育', '10':'体育'}
dic={1:'娱乐',2:'财经', 3:'时尚', 4:'房产', 5:'游戏', 6:'科技', 7:'家居', 8:'时政', 9:'教育', 10:'体育'}
lebal=[10]
a=lebal[0]
print(a)


print (dic[a])

10
体育


In [21]:
predict('我国拟建覆盖地震多发地区的实验性预警系统中新网5月11日电 中国地震局震害防御司副司长杜玮在国新办今天举行的新闻发布上透露，组织了大批的专家在这方面集中开展攻关工作，有望在近些年内就启动建设一个能够覆盖地震多发地区的实验性的地震预警系统。杜玮说，汶川地震以后，国务院也非常重视地震的预防，包括地震海啸预警系统的建设。杜玮介绍，所谓的实验性，主要是指这种预警系统建立起来以后，可能还不能马上做到对于每一次地震都提供出有效果的、对公众的避震和紧急防范都起到直接作用的预警信息，可能要有一个运行、实验、调整、完善的过程，但是有关方面的建设已经在酝酿之中。杜玮说，至于预警时间长短的问题，有赖于这个预警系统所针对的地震发生的目标，据了解，在海洋当中发生的地震，对大陆陆上人口比较密集城市和地区的影响的预警，可以做到预警时间相对长一些，能做到分钟级的，两三分钟甚至一两分钟的样子，如果陆地发生的地震，对震中地区的影响的预警，可以提供预警的时间可能只是以秒来计的，十几秒、几秒的时间。杜玮表示，尽管可以期望获取预警的时间可能是非常短暂的，但是，这种预警系统对于减少人员伤亡和减轻损失是非常有意义的，会不遗余力地推进这方面技术的发展，促进它在中国多地震地区能够得到应用。(据中国网文字直播整理)中国的减灾行动(全文)国新办发表《中国的减灾行动》白皮书')


1/1 [==============================] - 0s 102ms/step


'时政'

In [23]:
demo = gr.Interface(predict, inputs="text", outputs="text")
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://1e3d3c4e884f1bd81a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


1/1 [==============================] - 0s 25ms/step
